# Outline

Here we will train a fully-connected network to classify hand-written digits from images using the MNIST dataset

The network will take 28x28 pixel images and map them to digit categories {0, 1, 2, ..., 9}. 

In [ ]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms

# Load the Dataset (MNIST)


We can use some PyTorch DataLoader utilities for this.  

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=1000, shuffle=True)


# Display some images

In [ ]:
%run plot_conf.py

In [ ]:
plt_style()

In [ ]:
for i in range(10):
    plt.subplot(2, 5, i + 1)
    image, label = train_loader.dataset.__getitem__(i)
    plt.imshow(image.squeeze().numpy())
    plt.title(label)

# Running on a GPU: device string

Switching between CPU and GPU in PyTorch is controlled via a device string, which will seemlessly determine whether GPU is available, falling back to CPU if not:

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Create the model class (1 Layer Network)

In [ ]:
class Network1Layer(nn.Module):
    def __init__(self, input_size, n_hidden, output_size):
        super(Network1Layer, self).__init__()
        self.input_size = input_size
        self.layer1 = nn.Linear(input_size, n_hidden)
        self.layer2 = nn.Linear(n_hidden, n_hidden)
        self.layer3 = nn.Linear(n_hidden, output_size)

    def forward(self, x):
        x = x.view(-1, self.input_size)
        x = self.layer1(x)
        x = F.relu(x)
        x = self.layer2(x)
        x = F.relu(x)
        x = self.layer3(x)
        return F.log_softmax(x, dim=1)


# Define Training Loop

* Loop batches of samples in the training set
* Run each batch through the model (forward pass)
* Compute the loss
* Compute the gradients with respect to model parameters (backward pass)
* Update the parameters

In [ ]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device) #convert to CUDA tensors
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100 * batch_idx / len(train_loader), loss.item()))

# Define Testing Loop

* Loop over batches of samples in the testing set
* Run each batch through the model (forward pass)
* Compute the loss and accuracy
* Do not compute gradients or update model parameters 
* We are saving the testing data to evaluate how the model is doing on data it has not been trained on

In [ ]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device) #convert to CUDA tensors
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss                                                               
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability                                                                 
        correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100 * correct / len(test_loader.dataset)))

# Initialize the Model and Optimizer

In [ ]:
# Training settings 
input_size  = 28*28   # images are 28x28 pixels
n_hidden    = 100     # number of hidden units
output_size = 10      # there are 10 classes

model = Network1Layer(input_size, n_hidden, output_size)
model.to(device) #convert to CUDA model
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# Train the Model

* We will only train for a few epochs here
* Normally we would train for longer
* Depending on the dataset and model size, this can take days or weeks

In [ ]:
for epoch in range(1, 3):
    train(epoch)
    test()

# Now show some model predictions on testing data

* We will show an image from the testing set, and the probabilities the model assigns to each class

In [ ]:
# show an image and the predicted probabilities                                                                                                               
def display(i):
    plt.figure(i + 1)
    image, _ = test_loader.dataset.__getitem__(i)
    image = image.to(device) # convert to CUDA tensors
    logprob = model(image).cpu() #get tensors back to CPU for plotting
    prob = torch.exp(logprob).squeeze().data
    plt.subplot(121)
    plt.imshow(image.squeeze().numpy())
    plt.subplot(122)
    plt.bar(range(10), prob.numpy())
    plt.xlabel('predicted class probabilities')
    plt.xticks(range(10))
    plt.ylim([0, 1])

for i in range(10):
    display(i)

# What happens if we use less hidden units?

In [ ]:
# Training settings 
input_size  = 28*28   # images are 28x28 pixels
n_hidden    = 2     # number of hidden units
output_size = 10      # there are 10 classes

model = Network1Layer(input_size, n_hidden, output_size)
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [ ]:
for epoch in range(1, 3):
    train(epoch)
    test()

In [ ]:
for i in range(10):
    display(i)

# Things to try out

* Try different numbers of layers and hidden units
* Try different non-linearities (tanh, sigmoid)
* Try on other datasets (CIFAR 10)